In [22]:
# News Analysis Platform
# AI-driven financial news analysis using OpenAI and Perplexity APIs

import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env")

from perplexity import Perplexity
from openai import OpenAI
import pandas as pd
from datetime import datetime

# --- Load API Keys ---
openai_api_key = os.getenv("OPENAI_API_KEY")
perplexity_api_key = os.getenv("PERPLEXITY_API_KEY")

if not openai_api_key or not perplexity_api_key:
    raise EnvironmentError("Missing API keys in ../.env — please add OPENAI_API_KEY and PERPLEXITY_API_KEY")

# --- Initialize Clients ---
openai_client = OpenAI(api_key=openai_api_key)
pplx_client = Perplexity(api_key=perplexity_api_key)

print("OpenAI and Perplexity initialized successfully.")



OpenAI and Perplexity initialized successfully.


In [ ]:


class UtilsLLM:
    def __init__(self, perplexity_client=None, openai_client=None):
        self.pplx_client = perplexity_client
        self.openai_client = openai_client

    # ---------- PERPLEXITY ----------
    def call_perplexity(self, query, max_results=5):
        """Query the Perplexity API for a given search string."""
        if not self.pplx_client:
            raise ValueError("Perplexity client not initialized.")
        print(f"Searching Perplexity for: {query}")
        search = self.pplx_client.search.create(query=query, max_results=max_results)

        results = []
        for r in search.results:
            results.append({
                "title": r.title,
                "url": r.url,
                "snippet": r.snippet
            })
        return results

    # ---------- OPENAI ----------
    def call_openai(self, prompt_text, model="gpt-4o-mini", temperature=0.4):
        """Send prompt to OpenAI and return model response."""
        if not self.openai_client:
            raise ValueError("OpenAI client not initialized.")
        print("Calling OpenAI with custom prompt...")
        completion = self.openai_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a senior investment strategist."},
                {"role": "user", "content": prompt_text}
            ],
            temperature=temperature
        )
        return completion.choices[0].message.content



In [35]:

class PerplexitySearch:
    def __init__(self, llm_utils, base_dir="../../outputs/portfolio1"):
        self.llm_utils = llm_utils
        self.base_dir = base_dir
        self.analysis_dir = os.path.join(base_dir, "analysis")

    def compile_perplexity_report(self, symbols, max_results=5, custom_prompt=None):
        report_lines = []
        for sym in symbols[:10]:
            query = custom_prompt or f"Latest financial and market news for {sym} stock"
            try:
                news_items = self.llm_utils.call_perplexity(query, max_results=max_results)
                formatted_news = "\n".join(
                    [f"- {n['title']}: {n['url']}\n  {n['snippet']}" for n in news_items]
                )
                section = f"----- {sym} NEWS -----\n{formatted_news}\n"
                report_lines.append(section)
            except Exception as e:
                print(f"Error fetching news for {sym}: {e}")

        compiled_news = "\n".join(report_lines)

        # --- Save output ---
        date_str = datetime.now().strftime("%Y-%m-%d")
        news_dir = os.path.join(self.analysis_dir, "news")
        os.makedirs(news_dir, exist_ok=True)

        news_filename = f"perplexity_news_{date_str}.txt"
        news_path = os.path.join(news_dir, news_filename)

        with open(news_path, "w") as f:
            f.write(compiled_news)

        print(f"Perplexity news digest saved to {news_path}")
        return compiled_news, news_path

In [36]:
class PortfolioNewsAnalyzer(PerplexitySearch):
    def __init__(self, llm_utils, base_dir="../../outputs/portfolio1", analysis_prompt=None):
        super().__init__(llm_utils, base_dir)
        self.analysis_prompt = analysis_prompt or self.default_prompt()

    def default_prompt(self):
        with open("prompt.txt", "r", encoding="utf-8") as f:
            return f.read().strip()

    def load_portfolio_symbols(self):
        """Load stock symbols from the performance CSV."""
        path = os.path.join(self.base_dir, "data", "performance.csv")
        df = pd.read_csv(path, index_col=0)
        symbols = df.index.tolist()
        print(f"Loaded {len(symbols)} symbols from portfolio: {symbols}")
        return symbols

    def analyze_with_openai(self, news_digest, model="gpt-4o-mini", temperature=0.4):
        """Combine portfolio metrics + news digest and send to OpenAI."""
        llm_report_path = os.path.join(self.analysis_dir, "portfolio_llm_report.txt")
        with open(llm_report_path, "r") as f:
            portfolio_text = f.read()

        prompt_text = f"""
===== PORTFOLIO REPORT =====
{portfolio_text}

===== NEWS DIGEST =====
{news_digest}

{self.analysis_prompt}
"""

        analysis = self.llm_utils.call_openai(prompt_text, model=model, temperature=temperature)

        output_path = os.path.join(self.analysis_dir, "llm_digest.txt")
        with open(output_path, "w") as f:
            f.write(analysis)

        print(f"OpenAI analysis saved to {output_path}")
        return analysis, output_path


In [37]:


llm_utils = UtilsLLM(perplexity_client=pplx_client, openai_client=openai_client)

analyzer = PortfolioNewsAnalyzer(llm_utils)

symbols = analyzer.load_portfolio_symbols()

news_digest, news_path = analyzer.compile_perplexity_report(symbols, max_results=5)

analysis, output_path = analyzer.analyze_with_openai(news_digest)

print("\n============================================================")
print("                 PORTFOLIO LLM ANALYSIS SUMMARY             ")
print("============================================================")
print(f"Symbols analyzed: {symbols}")
print(f"News digest saved to: {news_path}")
print(f"OpenAI analysis saved to: {output_path}")
print(f"Generated on: {datetime.now().strftime('%m/%d/%Y %H:%M:%S')}")
print("------------------------------------------------------------\n")

# Optional: Print the first few lines of the analysis
print("\n===== PREVIEW =====\n")
print("\n".join(analysis.splitlines()[:30]))  # Preview top 30 lines
print("... [truncated]")


Loaded 17 symbols from portfolio: ['AAPL', 'MSFT', 'GOOGL', 'SPY', 'QQQ', 'VTI', 'TSLA', 'NVDA', 'JPM', 'BAC', 'VEA', 'VWO', 'JNJ', 'PFE', 'KO', 'PG', 'PORTFOLIO']
Searching Perplexity for: Latest financial and market news for AAPL stock
Searching Perplexity for: Latest financial and market news for MSFT stock
Searching Perplexity for: Latest financial and market news for GOOGL stock
Searching Perplexity for: Latest financial and market news for SPY stock
Searching Perplexity for: Latest financial and market news for QQQ stock
Searching Perplexity for: Latest financial and market news for VTI stock
Searching Perplexity for: Latest financial and market news for TSLA stock
Searching Perplexity for: Latest financial and market news for NVDA stock
Searching Perplexity for: Latest financial and market news for JPM stock
Searching Perplexity for: Latest financial and market news for BAC stock
Perplexity news digest saved to ../../outputs/portfolio1/analysis/news/perplexity_news_2025-10-19.tx